# Tutorial 4: Scheduling with CP: Some Basics 

Today we discover a new family of problems, called scheduling. Scheduling problems are widely present in real life applications such as timetabelling, transportation, project management, and manufacturing. We consider a particular problem called the job shop scheduling problem.  In this problem, we are given a set of $n$ jobs: $J_1, J_2, \ldots,  J_n$ and a set of $m$ machines $M_1, M_2, \ldots,  M_m$. 
Each job $J_i$ is defined by a set of $m$ (non-preemptive) tasks_duration $T_{i,1} \ldots T_{i,m}$. Every task $T_{i,k}$ is associated with a duration $p_{i,k}$ and is supposed to be scheduled on machine $M_k$. 

The problem has two sets of constraints: 

 - Precedence constraints: Each job is associated with an order of tasks_duration to respect when scheduling the different tasks_duration.
 - Disjunctive constraints: Each machine can process only one task at a given time


The standard optimisation version of this problem asks to minimize the makespan, i.e., the total scheduling time.


Constraint programming has been widely and successfully used to solve scheduling problems. Many global constraints have been proposed. CP solvers often offer a dedicated library for scheduling. 
Please have a look at the diffrent features proposed in docplex here http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html?highlight=scheduling#scheduling-functions 


In this tutorial, we will be using (only): 
 - Interval variables for the different variables of the problem:  http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.expression.py.html?highlight=interval_var#docplex.cp.expression.interval_var
 
 - end_before_start constraints to model precedence constraints : http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html?highlight=end_before_start#docplex.cp.modeler.end_before_start

- no_overlap global constraint : http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html?highlight=no_overlap#docplex.cp.modeler.no_overlap


The format for a job shop scheduling instance respects the following syntax: 
- The first line containts only $n$ $m$ in this order  ($n$ is the number of jobs and $m$ is the number of machines)
- Then $n$ lines are given. Each line $i$ is associated to the job $J_i$ and contains exactly $2 m$ integers $x^i_1$ $d^i_1$ $x^i_2$ $d^i_2$ $\ldots$ $x^i_n$ $d^i_n$. Each $x^i_k$ is the $k^{th}$ machine required by the $k^{th}$ task of the job $J_i$, and $d^i_k$ represents its duration. 

Consider for example the data file instance.data : 
    
 6  6
     
2   1  0   3  1   6  3   7  5   3  4   6

1   8  2   5  4  10  5  10  0  10  3   4

2   5  3   4  5   8  0   9  1   1  4   7
 
 1   5  0   5  2   5  3   3  4   8  5   9
 
 2   9  1   3  4   5  5   4  0   3  3   1
 
 1   3  3   3  5   9  0  10  4   4  2   1


This instance has $6$ jobs and $6$ machines. The first job requires the execution of task $T_{1,2}$ (on machine $2$) of duration $1$, 
then $T_{1,0}$ (on machine $0$) of duration $3$, etc. 

Write a simple python code to parse the data file instance.data

In [62]:
data = []

# Open the data file
with open("example.data", "r") as file:
    # Read the file line by line
    for line in file:
        # Strip any leading/trailing whitespace and split the line into individual values
        values = line.strip().split()
        
        # Convert each value to an integer and append it to the data list
        data.extend(map(int, values))

# Print the parsed data
print(data)


[6, 6, 2, 1, 0, 3, 1, 6, 3, 7, 5, 3, 4, 6, 1, 8, 2, 5, 4, 10, 5, 10, 0, 10, 3, 4, 2, 5, 3, 4, 5, 8, 0, 9, 1, 1, 4, 7, 1, 5, 0, 5, 2, 5, 3, 3, 4, 8, 5, 9, 2, 9, 1, 3, 4, 5, 5, 4, 0, 3, 3, 1, 1, 3, 3, 3, 5, 9, 0, 10, 4, 4, 2, 1]


In [63]:
n, m = data[:2]
print("The number of jobs is : ", n)
print("The number of machines is : ", m)
data_tasks_duration = data[2:]
print(data_tasks_duration)


The number of jobs is :  6
The number of machines is :  6
[2, 1, 0, 3, 1, 6, 3, 7, 5, 3, 4, 6, 1, 8, 2, 5, 4, 10, 5, 10, 0, 10, 3, 4, 2, 5, 3, 4, 5, 8, 0, 9, 1, 1, 4, 7, 1, 5, 0, 5, 2, 5, 3, 3, 4, 8, 5, 9, 2, 9, 1, 3, 4, 5, 5, 4, 0, 3, 3, 1, 1, 3, 3, 3, 5, 9, 0, 10, 4, 4, 2, 1]


Create a matrix machine that satisfies: machine[i][k] is the machine of the $k^{th}$ task of job $i$ (one line)

In [64]:
import numpy as np
machine = np.zeros((n,m))
# print(jobs)

for i in range(n):
    for j in range(0,m):
        k = 2*j
        machine[i,j] = data_tasks_duration[i*2*m+k]

print(machine)

[[2. 0. 1. 3. 5. 4.]
 [1. 2. 4. 5. 0. 3.]
 [2. 3. 5. 0. 1. 4.]
 [1. 0. 2. 3. 4. 5.]
 [2. 1. 4. 5. 0. 3.]
 [1. 3. 5. 0. 4. 2.]]


Create a matrix duration that satisfies: duration[i][k] is the duration of the $k^{th}$ task of job $i$ (one line)

In [65]:
duration = np.zeros((n,m))
# print(jobs)

for i in range(n):
    for j in range(0,m):
        k = 2*j
        duration[i,j] = data_tasks_duration[i*2*m+k+1]

print(duration)

[[ 1.  3.  6.  7.  3.  6.]
 [ 8.  5. 10. 10. 10.  4.]
 [ 5.  4.  8.  9.  1.  7.]
 [ 5.  5.  5.  3.  8.  9.]
 [ 9.  3.  5.  4.  3.  1.]
 [ 3.  3.  9. 10.  4.  1.]]


Compute a naive upper bound for the makespan. Note: this upper bound will be used as an upper bound for every interval variable we create.

In [66]:
# Compute the naive upper bound (maximum sum of durations across all jobs)
naive_upper_bound = np.sum(duration)
# Print the naive upper bound
print("Naive Upper Bound:", naive_upper_bound)

Naive Upper Bound: 197.0


Create a CpoModel() and the different interval variables you need to solve this problem (don't forget to use the upper bound you computed earlier)

In [67]:
from config import setup
setup()

## install docplex first with $pip install docplex
from docplex.cp.model import *
from docplex.cp.config import get_default

In [68]:
#Create a CpoModel() 
mdl = CpoModel()

#Create task duration
tasks_duration = [[mdl.interval_var(size=int(duration[i][j]), name="T{}{}".format(i,j)) for i in range(n)] for j in range(m)]

# Time of starting each task
S = mdl.integer_var_list(n*m, 0, int(naive_upper_bound), "marks")

Post the precedence constraints using the end_before_start constraints 

In [69]:
#Add end_before_start constraints

# For each job
for job_i in range(n):

    # For each machine 
    for machine_i in range(1,m):
        
        # Plusieurs machines ne peuvent pas effectuer le même job en même temps
        mdl.add(mdl.end_before_start(tasks_duration[job_i][machine_i],tasks_duration[job_i][machine_i-1]))

Post the disjunctive constraints using the no-overlap constraints 

In [70]:
#Add no-overlap constraints

#For each machine
for machine_i in range(m):

    #For each job
    for job_i in range(n-1):
        # Plusieurs jobs ne peuvent pas être effectués en même temps sur la même machine
        mdl.add(mdl.no_overlap([tasks_duration[job_i][machine_i],tasks_duration[job_i+1][machine_i]]))

Create a makespan interval variable and link it to some variables using the max global constraint
http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html?highlight=max#docplex.cp.modeler.max

In [71]:
#Create makespan interval variable
makespan = mdl.interval_var(end=max([tasks_duration[i][m-1].end for i in range(n)]), name="makespan")

#On cherche à minimiser la fin de la dernière tâche à l'aide d'une contrainte max globale

Add now the makespan as an objective function 

In [72]:
mdl.add(minimize(end_of(makespan)))

Solve this instance. What is the value of the makespan you found. You can print the solution in a format that is easy 
to see visually.  

In [75]:
#Solve this instance
msol = mdl.solve(TimeLimit=10)
# msol = mdl.start_search(SearchType="DepthFirst")

# j=0
# for i in msol:
#     j += 1
#     i.write()
# print(j)

 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Minimization problem - 67 variables, 60 constraints
 ! Presolve             = Off
 ! TimeLimit            = 10
 ! Workers              = 1
 ! Initial process time : 0.02s (0.02s extraction + 0.00s propagation)
 !  . Log search space  : 198.0 (before), 198.0 (after)
 !  . Memory usage      : 534.6 kB (before), 534.6 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed            Branch decision
                        0         67                 -
 + New bound is 0
 *             0       15  0.07s               (gap is 0.00%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : 0 (optimal - effective tol. is 0)
 ! Best bound             : 0
 ! --------------------------------------------------------------------

In [76]:
#Print in a format that is easy to see visually

for i in range(m):
    print("Machine ", i, ": ", end="")
    for j in range(n):
        print("Job ", j, ": ", msol.get_var_solution(tasks_duration[i][j]), " ", end="")
    print()


Machine  0 : Job  0 :  T00: (start=30, end=31, size=1, length=1)  Job  1 :  T10: (start=22, end=30, size=8, length=8)  Job  2 :  T20: (start=17, end=22, size=5, length=5)  Job  3 :  T30: (start=12, end=17, size=5, length=5)  Job  4 :  T40: (start=3, end=12, size=9, length=9)  Job  5 :  T50: (start=0, end=3, size=3, length=3)  
Machine  1 : Job  0 :  T01: (start=35, end=38, size=3, length=3)  Job  1 :  T11: (start=30, end=35, size=5, length=5)  Job  2 :  T21: (start=22, end=26, size=4, length=4)  Job  3 :  T31: (start=17, end=22, size=5, length=5)  Job  4 :  T41: (start=12, end=15, size=3, length=3)  Job  5 :  T51: (start=3, end=6, size=3, length=3)  
Machine  2 : Job  0 :  T02: (start=45, end=51, size=6, length=6)  Job  1 :  T12: (start=35, end=45, size=10, length=10)  Job  2 :  T22: (start=27, end=35, size=8, length=8)  Job  3 :  T32: (start=22, end=27, size=5, length=5)  Job  4 :  T42: (start=15, end=20, size=5, length=5)  Job  5 :  T52: (start=6, end=15, size=9, length=9)  
Machine 

Factorise your code so that it takes as input the data file and it solves the problem. Try to use few other instances https://github.com/tamy0612/JSPLIB/tree/master/instances

In [124]:
#Factorise the code so that it takes as input the data file and solves the problem
import numpy as np
from config import setup
setup()

# install docplex first with $pip install docplex
from docplex.cp.model import *
from docplex.cp.config import get_default


def solve(f):
    data = []
    with open(f, "r") as file:
        # Read the file line by line
        for line in file:
            # Strip any leading/trailing whitespace and split the line into individual values
            values = line.strip().split()
            
            # Convert each value to an integer and append it to the data list
            data.extend(map(int, values))

    n, m = data[:2]
    print("The number of jobs is : ", n)
    print("The number of machines is : ", m)
    data_tasks_duration = data[2:]
    print("All jobs, where each task is succeeded by its duration is : ", data_tasks_duration)

    machine = np.zeros((n,m))

    for i in range(n):
        for j in range(0,m):
            k = 2*j
            machine[i,j] = data_tasks_duration[i*2*m+k]

    print("\n Just below, the matrix Machine such as machine[i][k] is the machine of the k_th task of job i \n" , machine)

    duration = np.zeros((n,m))

    for i in range(n):
        for j in range(0,m):
            k = 2*j
            duration[i,j] = data_tasks_duration[i*2*m+k+1]

    print("\n Just below, the matrix Duration such as duration[i][k] is the duration of the k_th task of job i \n", duration)

    # Compute the naive upper bound (maximum sum of durations across all jobs)
    naive_upper_bound = np.sum(duration)
    # Print the naive upper bound
    print("\n A Naive Upper Bound: would be : ", naive_upper_bound)

    #Create a CpoModel() 
    mdl = CpoModel()

    #Create task duration
    tasks_duration = [[mdl.interval_var(size=int(duration[i][j]), name="T{}{}".format(i,j)) for i in range(n)] for j in range(m)]

    # Time of starting each task
    S = mdl.integer_var_list(n*m, 0, int(naive_upper_bound), "time_starting")

    #Variable machine
    M = mdl.integer_var_list(m, 0, m-1, "machine")

    #Associate machine with its task
    # for i in range(n):
    #     for j in range(m):
    #         if (j != int(machine[i][j])):
    #             mdl.add(start_of(tasks_duration[i][int(machine[i][j])]) != start_of(tasks_duration[i][j]))
    #             mdl.add(end_of(tasks_duration[i][int(machine[i][j])]) != end_of(tasks_duration[i][j]))
    #         else:
    #             mdl.add(start_of(tasks_duration[i][int(machine[i][j])]) == start_of(tasks_duration[i][j]))
    #             mdl.add(end_of(tasks_duration[i][int(machine[i][j])]) == end_of(tasks_duration[i][j]))


    #Add end_before_start constraints

    # For each job
    for job_i in range(n):

        # For each machine 
        for machine_i in range(1,m):
            
            # Plusieurs machines ne peuvent pas effectuer le même job en même temps
            mdl.add(mdl.end_before_start(tasks_duration[job_i][machine_i],tasks_duration[job_i][machine_i-1]))


    #Add no-overlap constraints

    #For each machine
    for machine_i in range(m):

        #For each job
        for job_i in range(n-1):
            # Plusieurs jobs ne peuvent pas être effectués en même temps sur la même machine
            mdl.add(mdl.no_overlap([tasks_duration[job_i][machine_i],tasks_duration[job_i+1][machine_i]]))

    
    #Create makespan interval variable
    makespan = mdl.interval_var(end=max([tasks_duration[i][m-1].end for i in range(n)]), name="makespan")


    #On cherche à minimiser la fin de la dernière tâche à l'aide d'une contrainte max globale

    mdl.add(minimize(end_of(makespan)))

    #Solve this instance
    msol = mdl.solve(TimeLimit=10)

    #Print in a format that is easy to see visually

    for i in range(m):
        print("Machine ", i, ": ", end="")
        for j in range(n):
            print("Job ", j, ": ", msol.get_var_solution(tasks_duration[i][j]), " ", end="")
        print()



solve("example3.data")

The number of jobs is :  2
The number of machines is :  2
All jobs, where each task is succeeded by its duration is :  [1, 20, 0, 5, 0, 5, 1, 18]

 Just below, the matrix Machine such as machine[i][k] is the machine of the k_th task of job i 
 [[1. 0.]
 [0. 1.]]

 Just below, the matrix Duration such as duration[i][k] is the duration of the k_th task of job i 
 [[20.  5.]
 [ 5. 18.]]

 A Naive Upper Bound: would be :  48.0
 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Minimization problem - 7 variables, 12 constraints
 ! Presolve             = Off
 ! TimeLimit            = 10
 ! Workers              = 1
 ! Initial process time : 0.02s (0.02s extraction + 0.00s propagation)
 !  . Log search space  : 13.9 (before), 13.9 (after)
 !  . Memory usage      : 302.7 kB (before), 302.7 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed            Branch de

At this stage you are completly free to play. Try different instances, different configurations of the solver, different branching strategies, restarts, randomisation, etc. 

You may want to present your results as a cactus 🌵 plot : in the x-axis we have the runtime, on the y-axis, we have the number of instances solved. Also, some instances are still open in the literature. Have a look here for an up to date list of bounds http://optimizizer.com/TA.php

What did you learn loday? 